# From Google Maps coordinates to a planar coordinates

The UI could be based on Google Maps to indicate (or even automatically retrieve) the coordinates of the points. This requires to be able to translate coordinates in DMS to UTM.

## Todo:

 * Don't forget to give a try to geographiclib...
 * Also check https://pypi.python.org/pypi/utm

In [3]:
from math import sqrt, sin, cos, tan, pi
import utm

Google maps returns locations in the systeme geodesique WGS84 using the DMS notation:

In [4]:
# Location of our home:
location = '''+43°37′3.66",+7°4′43.08"'''

Let's convert that to the DD notation:

In [5]:
import re
dms_re = re.compile("([+-])\s*(\d+)[^\d]+\s*(\d+)[^\d]+\s*([\d.]+)[^\d]+,\s*([+-])\s*(\d+)[^\d]+\s*(\d+)[^\d]+\s*([\d.]+)[^d]+")

def dms_to_dd(s):
    """Convert a string of the form +43°37′3.66",+7°4′43.08" (DMS notation) to a couple of float (DD notation)."""
    m = dms_re.search(s)
    latitude = float(m.group(2)) + float(m.group(3))/60 + float(m.group(4))/(60**2)
    if m.group(1) == '-':
        latitude = -latitude
    longitude = float(m.group(6)) + float(m.group(7))/60 + float(m.group(8))/(60**2)
    if m.group(5) == '-':
        longitude = - longitude
    return (latitude, longitude)


(lat, lon) = dms_to_dd(location)
(lat, lon)


(43.61768333333333, 7.078633333333333)

Let's consider we are in France and take the reference meridian phi0 = 7 degrees.

In [6]:
lon0 = 7.

Here is a function to convert from DD coordinates in the wsg84 geodesic system to UTM coordinates.

(ref: http://fr.wikipedia.org/wiki/Transverse_Universelle_de_Mercator).

Note: at some point, it might be worth to check the english page of Wikipedia that also comes with the reverse operation.

In [8]:
def wsg84_to_umt(lat, lon, lon0=None):
    """
    Convert from WSG84 coordinates in the geodesic system in DD format (e.g. (lat, lon) in degrees) to UTM coordinates (x, y) in metres.
    The reference meridian is given in lon0.
    """
    if lon0 is None:
        lon0 = float(int(lon))
    # convert from degree to radian
    lat = 2.*pi*lat/360.
    lon = 2.*pi*lon/360.
    lon0 = 2.*pi*lon0/360.
    # ref of the calculus below: http://fr.wikipedia.org/wiki/Transverse_Universelle_de_Mercator
    e = 0.0818192
    a = 6378.137
    v = 1. / sqrt(1. - e**2 * sin(lat)**2)
    A = (lon-lon0) * cos(lat)
    s = (1. - e**2/4. - 3*e**4/64. - 5*e**6/256.) * lat
    s -= (3*e**2/8. + 3*e**4/32. + 45*e**6/1024.) * sin(2.*lat)
    s += (15.*e**4/256. + 45*e**6/1024.) * sin(4.*lat)
    s -= (35*e**6/3072.) * sin(6*lat)
    T = tan(lat)**2
    C = e**2 / (1.-e**2) * cos(lat)**2
    k0 = 0.9996
    if(lat >= 0):
        N0 = 0.
    else:
        N0 = 10000
    E = 500. + k0*a*v*(A + (1.-T+C)*A**3/6. + (5-18*T+T**2)*A**5/120.)
    N = N0 + k0*a*(s+v*tan(lat)*(A**2/2+(5-T+9*C+4*C**2)*A**4/24+(61-58*T+T**2)*A**6/720))
    return (E*1000, N*1000)

wsg84_to_umt(lat, lon, lon0)

(506344.6985955992, 4829414.195055106)

Let's check with the example from wikipedia's page:
(ref: http://fr.wikipedia.org/wiki/Transverse_Universelle_de_Mercator)

In [10]:
(lat, lon) = dms_to_dd('''+45°09′43.08",+5°50′51"''')
(e, n) = wsg84_to_umt(lat, lon, 3.)
(723792, 5004888) == (int(e), int(n))

True

In [13]:
wsg84_to_umt(*dms_to_dd(location))

(506344.6985955992, 4829414.195055106)

Let's compare measure from IGN map and GPS coordinates from wikipedia.

In [16]:
points = [
    ("Aiguille du Midi", (13*40+2, 11*40+29), 35.5, "+45° 52′ 43″, +6° 53′ 14″"),
    ("Mont Blanc du Tacul", (13*40+1, 9*40+11), 45.0, "+45° 51′ 24″, +6° 53′ 16″"),
    ("Mont Maudit", (12*40+3, 8*40+12), 56.3, "+45° 50′ 52″, +6° 52′ 33″"),
    ("Mont Blanc", (11*40+5, 6*40+27),  65.8, "+45° 49′ 57″, +6° 51′ 53″"),
    ("Dome du Gouter",  (9*40+21, 7*40+33),  80.4, "+45° 50′ 34″, +6° 50′ 36″"),
    ("Aiguille du Gouter",  (8*40+25, 8*40+3),  92.3, "+45° 51′ 02″, +6° 49′ 52″"),
    ("Aiguille de Bionnassay",  (7*40+23, 7*40+6),  97.2, "+45° 50′ 09″, +6° 49′ 05″")
]

Let's have both coordinates together with same scalre (meters).

In [18]:
coords = [((p[1][0]*25, p[1][1]*25), wsg84_to_umt(*dms_to_dd(p[3]))) for p in points]
coords

[((13050, 11725), (568849.6664404997, 5080943.178602797)),
 ((13025, 9275), (568919.894701822, 5078505.544897098)),
 ((12075, 8300), (568003.4598252792, 5077507.715586104)),
 ((11125, 6675), (567159.1268773016, 5075800.900087536)),
 ((9525, 7825), (565485.8923633278, 5076925.033257393)),
 ((8625, 8075), (564527.8291993187, 5077779.221778985)),
 ((7575, 7150), (563530.9482673384, 5076133.063705153))]

Let's correct the fact that the origin is not the same.

In [19]:
x0 = coords[0][1][0]-coords[0][0][0]
y0 = coords[0][1][1]-coords[0][0][1]
(x0, y0)

(555799.6664404997, 5069218.178602797)

In [20]:
corrected_coord = [((c[0][0]+x0, c[0][1]+y0), c[1]) for c in coords]
corrected_coord

[((568849.6664404997, 5080943.178602797),
  (568849.6664404997, 5080943.178602797)),
 ((568824.6664404997, 5078493.178602797),
  (568919.894701822, 5078505.544897098)),
 ((567874.6664404997, 5077518.178602797),
  (568003.4598252792, 5077507.715586104)),
 ((566924.6664404997, 5075893.178602797),
  (567159.1268773016, 5075800.900087536)),
 ((565324.6664404997, 5077043.178602797),
  (565485.8923633278, 5076925.033257393)),
 ((564424.6664404997, 5077293.178602797),
  (564527.8291993187, 5077779.221778985)),
 ((563374.6664404997, 5076368.178602797),
  (563530.9482673384, 5076133.063705153))]

Errors on X and Y

In [21]:
errors = [(cc[1][0]-cc[0][0], cc[1][1]-cc[0][1])for cc in corrected_coord]
errors

[(0.0, 0.0),
 (95.22826132224873, 12.366294301114976),
 (128.7933847794775, -10.46301669254899),
 (234.46043680189177, -92.27851526066661),
 (161.22592282807454, -118.14534540381283),
 (103.16275881894398, 486.04317618813366),
 (156.28182683873456, -235.11489764414728)]

Distance between the two coordinates (in metres).

In [22]:
[sqrt(e[0]**2+e[1]**2) for e in errors]

[0.0,
 96.02784486387417,
 129.21768718431355,
 251.96630886697326,
 199.88026624044326,
 496.87073160549994,
 282.31724087322414]

Erros are big but not so big...
Meaning, the computations done above "make sense" but are not very precise.
Next step is probably to check UTM computation done here vs what's on the IGN map.

In [25]:
mtblanc = points[3]
mtblancdd = dms_to_dd(mtblanc[3])
print(wsg84_to_umt(*mtblancdd, 9))

(334163.36666377855, 5077654.467275109)


In [26]:
utm.from_latlon(mtblancdd[0], mtblancdd[1])

(334163.3668251076, 5077654.472746319, 32, 'T')

In [27]:
mtblanc

('Mont Blanc', (445, 267), 65.8, '+45° 49′ 57″, +6° 51′ 53″')

In [28]:
1./8.

0.125

In [29]:
27./40.

0.675